In [1]:
import pandas as pd

data = pd.read_parquet("../pipelines/data/db_pull_calibrated.parquet")
data

,timestamp,Accelerometer_x,Accelerometer_y,Accelerometer_z,Gyroscope_x,Gyroscope_y,Gyroscope_z,Magnetometer_x,Magnetometer_y,Magnetometer_z,activity,hash,person
0,2023-03-16 15:50:20.660,0.856745,-9.736820,0.020757,0.268116,-1.078746,1.059684,-17.852205,35.951828,-0.945442,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
1,2023-03-16 15:50:20.670,0.909825,-8.220062,1.002787,0.271234,-1.028209,1.029722,-17.822435,35.816177,-1.256371,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
2,2023-03-16 15:50:20.680,0.336442,-7.226947,2.845873,0.084436,-0.911180,0.878822,-17.261415,35.596741,-1.602486,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
3,2023-03-16 15:50:20.690,-0.488674,-6.452185,5.054952,-0.314842,-0.878484,0.670179,-17.013292,35.198273,-1.386124,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
4,2023-03-16 15:50:20.700,-1.977320,-5.265067,5.953660,-0.817032,-0.921044,0.375689,-16.602587,35.248901,-1.095116,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4314652,2023-03-12 13:25:03.670,8.873645,-12.125825,7.280043,-1.097083,-2.545262,-3.277552,-32.639999,26.099998,23.580000,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess
4314653,2023-03-12 13:25:03.680,3.578034,-10.812859,7.249442,-0.878516,-3.433582,-2.865095,-33.480000,24.660000,24.240000,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess
4314654,2023-03-12 13:25:03.690,-0.935012,-8.310925,6.780705,-0.698311,-3.857278,-2.475241,-33.480000,23.400000,25.379999,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess
4314655,2023-03-12 13:25:03.700,-4.368380,-5.661316,7.183708,-0.454698,-3.775789,-2.106645,-34.380001,21.660000,26.760000,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess


In [2]:
data.value_counts(["hash", "activity", "person"]).head(10)

hash                              activity    person         
31ee3891f5ad49b1a989abec4dd30816  Sitzen      Ognjen_Colovic     121884
44c4e5f5f7df4203aca2f4e17afc6f35  Velofahren  Gabriel_Torres     105252
fc4955e4028d4e568ea29e35f8dfbeb9  Laufen      Ognjen_Colovic     100446
a212351e846d4da099db1a73032efe43  Stehen      Florin_Barbisch     95213
517458b7423a4f36920d4d95c53ad540  Sitzen      Yvo_Keller          95001
4eb98a019979402c87f4e0038a8ccf45  Laufen      Florin_Barbisch     81102
1c209f6ac3a546c498ca73ebb195611a  Sitzen      Florin_Barbisch     66210
e5236dad903942eabf03264e26c6d40f  Rennen      Florin_Barbisch     63825
a62bc673935e435ba7be437144e6827e  Sitzen      Tobias_Buess        63718
2c739c059b9f4632966a704e6970905e  Sitzen      Gabriel_Torres      61886
Name: count, dtype: int64

In [3]:
data = data[data["hash"] == "2c739c059b9f4632966a704e6970905e"].copy()
data = data[
    [
        "timestamp",
        "Accelerometer_x",
        "Accelerometer_y",
        "Accelerometer_z",
        "Gyroscope_x",
        "Gyroscope_y",
        "Gyroscope_z",
        "Magnetometer_x",
        "Magnetometer_y",
        "Magnetometer_z",
    ]
].copy()

In [4]:
# send data via api request
import requests
import gzip
import json

# prepare data
method = "POST"
url = "https://sbar.fuet.ch/CNN"
payload = data.to_csv(float_format="%.3f", index=False).encode("utf-8")
payload = gzip.compress(payload)
print(f"Payload Size: {len(payload) / 1000000} MB")

# send request
response = requests.request(method, url, data=payload)
text = response.content.decode("utf-8")
text = json.dumps(json.loads(text), indent=2)

# print response
print(f"Response Code: {response.status_code}")
print(text)

Payload Size: 1.083163 MB
Response Code: 200
{
  "0": "Sitzen",
  "1": "Sitzen",
  "2": "Sitzen",
  "3": "Sitzen",
  "4": "Sitzen",
  "5": "Sitzen",
  "6": "Sitzen",
  "7": "Sitzen",
  "8": "Sitzen",
  "9": "Sitzen",
  "10": "Sitzen",
  "11": "Sitzen",
  "12": "Sitzen",
  "13": "Sitzen",
  "14": "Sitzen",
  "15": "Sitzen",
  "16": "Sitzen",
  "17": "Sitzen",
  "18": "Sitzen",
  "19": "Sitzen",
  "20": "Sitzen",
  "21": "Sitzen",
  "22": "Sitzen",
  "23": "Sitzen",
  "24": "Sitzen",
  "25": "Sitzen",
  "26": "Sitzen",
  "27": "Sitzen",
  "28": "Sitzen",
  "29": "Sitzen",
  "30": "Sitzen",
  "31": "Sitzen",
  "32": "Sitzen",
  "33": "Sitzen",
  "34": "Sitzen",
  "35": "Sitzen",
  "36": "Sitzen",
  "37": "Sitzen",
  "38": "Sitzen",
  "39": "Sitzen",
  "40": "Sitzen",
  "41": "Sitzen",
  "42": "Sitzen",
  "43": "Sitzen",
  "44": "Sitzen",
  "45": "Sitzen",
  "46": "Sitzen",
  "47": "Sitzen",
  "48": "Sitzen",
  "49": "Sitzen",
  "50": "Sitzen",
  "51": "Sitzen",
  "52": "Sitzen",
  "53": "